In [1]:

import pandas as pd
from pathlib import Path

excel_path = Path(r"/Users/cheickberthe/PycharmProjects/spreadsheet-analyzer/test_assets/collection/business-accounting/Business Accounting.xlsx")
df = pd.read_excel(excel_path, sheet_name=1)
print(f"Loaded sheet with shape: {df.shape}")

# Quick preview for LLM context
print("\nFirst 10 rows:")
print(df.head(10).to_string())
print("\nLast 5 rows:")
print(df.tail(5).to_string())

# Get basic info
sheet_name = "None"
sheet_dimensions = f"{df.shape[0]} rows x {df.shape[1]} columns"


Loaded sheet with shape: (769, 11)

First 10 rows:
        Date                           Description  Transaction type   USD Amount     XOF Amount   Unnamed: 5              Category   TOTAL REVENUES   84202.53   TOTAL EXPENSES  -385526.333434
0 2023-01-03                        iCloud Storage               EXP        -9.99   -6214.329474   622.055002                   NaN              NaN        NaN              NaN             NaN
1 2023-01-03                   Amazon web services               EXP      -284.83 -177179.926325   622.055002                   NaN              NaN        NaN              NaN             NaN
2 2023-01-03                Diesel for return trip               EXP      -526.48    -327,500CFA          NaN                  Fuel              NaN        NaN              NaN             NaN
3 2023-01-03                         Road expenses               EXP      -120.57     -75,000CFA          NaN             Toll Fees              NaN        NaN              NaN 

In [2]:

excel_file_name = excel_path.name
sheet_dimensions = f"{df.shape[0]} rows x {df.shape[1]} columns"
sheet_name


'None'

In [3]:

# Table Detection Analysis
print("=== DATA STRUCTURE ANALYSIS ===")
print(f"DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\n=== FIRST 10 ROWS ===")
print(df.head(10).to_string())
print("\n=== LAST 5 ROWS ===")
print(df.tail(5).to_string())

print("\n=== COLUMN ANALYSIS ===")
# Check for empty columns that might separate tables
for i, col in enumerate(df.columns):
    non_null_count = df[col].notna().sum()
    print(f"Column {i} ('{col}'): {non_null_count}/{len(df)} non-null values")

print("\n=== EMPTY ROW ANALYSIS ===")
# Check for empty rows that might separate tables
empty_rows = df.isnull().all(axis=1)
if empty_rows.any():
    empty_row_indices = empty_rows[empty_rows].index.tolist()
    print(f"Empty rows found at indices: {empty_row_indices[:10]}")
else:
    print("No completely empty rows found")

print("\n=== SIDE-BY-SIDE ANALYSIS ===")
# Check for potential side-by-side tables by examining column groups
left_cols = df.iloc[:, 0:7]  # First 7 columns
right_cols = df.iloc[:, 7:]  # Remaining columns
print(f"Left section shape: {left_cols.shape}")
print(f"Right section shape: {right_cols.shape}")
print(f"Left section non-null density: {left_cols.notna().sum().sum() / (left_cols.shape[0] * left_cols.shape[1]):.2f}")
print(f"Right section non-null density: {right_cols.notna().sum().sum() / (right_cols.shape[0] * right_cols.shape[1]):.2f}")


=== DATA STRUCTURE ANALYSIS ===
DataFrame shape: (769, 11)
Columns: ['Date', 'Description', 'Transaction type', 'USD Amount', 'XOF Amount', 'Unnamed: 5', 'Category', 'TOTAL REVENUES', 84202.53000000001, 'TOTAL EXPENSES', -385526.33343393693]

=== FIRST 10 ROWS ===
        Date                           Description  Transaction type   USD Amount     XOF Amount   Unnamed: 5              Category   TOTAL REVENUES   84202.53   TOTAL EXPENSES  -385526.333434
0 2023-01-03                        iCloud Storage               EXP        -9.99   -6214.329474   622.055002                   NaN              NaN        NaN              NaN             NaN
1 2023-01-03                   Amazon web services               EXP      -284.83 -177179.926325   622.055002                   NaN              NaN        NaN              NaN             NaN
2 2023-01-03                Diesel for return trip               EXP      -526.48    -327,500CFA          NaN                  Fuel              NaN        

In [4]:
detected_tables = []

# Based on the analysis, the main data appears to be a single table spanning from row 0 to 768 and columns 0 to 10.
# The side-by-side analysis indicates a clear separation between two sections: left (columns 0-6) and right (columns 7-10).
# The left section contains detailed transaction data, while the right section appears to be summary or additional info, but with no non-null data.

# Main transaction table
detected_tables.append({
    "table_id": "table_1",
    "description": "Main transaction data with detailed records",
    "start_row": 0,
    "end_row": 768,
    "start_col": 0,
    "end_col": 10,
    "confidence": 0.95,
    "table_type": "DETAIL",
    "entity_type": "transactions"
})

# No other distinct tables are clearly identified from the side-by-side analysis or headers.
# The right section (columns 7-10) has no data, indicating it might be part of the main table or empty.

# If there were headers or other tables, they would be identified here.

# Final list
detected_tables = detected_tables

In [5]:

# Extract detection results created by LLM
# The detector should have created a 'detected_tables' variable with the results

# CLAUDE-TEST-WORKAROUND: Validate that detected_tables exists and is properly formatted
if 'detected_tables' in globals():
    # Validate it's a list
    if isinstance(detected_tables, list):
        detection_results = detected_tables
        print(f"✅ Found {len(detection_results)} tables from LLM detection")
        # Validate first table has required fields (if any tables exist)
        if detection_results:
            required_fields = ['table_id', 'description', 'start_row', 'end_row', 'start_col', 'end_col']
            first_table = detection_results[0]
            missing_fields = [f for f in required_fields if f not in first_table]
            if missing_fields:
                print(f"⚠️ Warning: First table missing fields: {missing_fields}")
    else:
        print(f"❌ Error: detected_tables is not a list, it's a {type(detected_tables)}")
        detection_results = []
else:
    # CLAUDE-GOTCHA: Gemini sometimes fails to create the variable even after multiple prompts
    print("❌ No 'detected_tables' variable found - LLM failed to complete detection")
    detection_results = []

detection_results


✅ Found 1 tables from LLM detection


[{'table_id': 'table_1',
  'description': 'Main transaction data with detailed records',
  'start_row': 0,
  'end_row': 768,
  'start_col': 0,
  'end_col': 10,
  'confidence': 0.95,
  'table_type': 'DETAIL',
  'entity_type': 'transactions'}]


# Table Detection Results

Detected 1 tables:


## Table 1: Main transaction data with detailed records
- Location: Rows 0-768, Columns 0-10
- Type: detail
- Entity: transactions
- Confidence: 0.95
